# Model Management


### 0. [Prerequisite] Install a dataset and a model from an existing dpk on the project

Get the current project

In [ ]:
import dtlpy as dl

# project_id is automatically set by the notebook 
project_id = 'project_id'

project = dl.projects.get(project_id=project_id)

## Install the app from dpk with the dataset and the model

A Model (architecture) entity in Dataloop refers to a machine learning algorithm that has been trained on labeled data to make predictions or perform other Tasks. Models are a key component of the Dataloop system, as they enable users to apply machine-learning techniques to a wide range of data types, including images, text, and audio. A Dataloop Model is a combination of data configurations and code to represent a learnable instance of the data.



In [10]:

dpk = project.dpks.get(dpk_name="ml-compare-example")
app = project.apps.install(dpk=dpk)

[2024-09-09 17:10:26][ERR][dtlpy:v1.91.37][services.api_client:1354] [Response <404>][Reason: Not Found][Text: {"message":"(traceId:6a238db300aa005ed5e297f08ee45806/14033512225059301426) Failed to get project 32e17d45-76a4-4506-a7b3-068d142586b2. Error:  not found, Project 32e17d45-76a4-4506-a7b3-068d142586b2 not found, Request failed with status code 404"}]


NotFound: ('404', '(traceId:6a238db300aa005ed5e297f08ee45806/14033512225059301426) Failed to get project 32e17d45-76a4-4506-a7b3-068d142586b2. Error:  not found, Project 32e17d45-76a4-4506-a7b3-068d142586b2 not found, Request failed with status code 404')

## Get the dataset after the app is installed

A dataset is a bucket collection of Items (files), their metadata and annotations. It can have a file-system-like structure with folders and subfolders at any level. A dataset is mapped to a Driver (which derives from an Integration), to contain items synced from external cloud storage. Dataset versioning actions include cloning and merging.

In [ ]:
dataset = project.datasets.get(dataset_name="V2 Plant Seedlings Dataset")

### 1. Model Get

In [ ]:
model = project.models.get(model_name='pretrained-resnet')

### 2. Model Clone

To create a new copy of our existing model, run the following command
<!-- why clone? explain more?, deploy, clone  then train t -->

In [ ]:
clone_model = model.clone(model_name='my pretrained resnet',
                          description='pretrained cloned into my project',
                          project_id=project_id)

### 3. Model Deploy

In case that the model is already ready for production, deploy it by the following command

In [ ]:
model.deploy()

### 4. Model Train

Notice: Deployed models cannot be trained, so in this step and the next ones, the clone model that was created on [step 2: clone a model](model_management.ipynb#2-clone-a-model) will be used.

To train a model on a dataset we need to:
1. Connect a dataset to the model, that will be used for the training.
2. Add the train and validation filters to the model, that will help to split the data to train and validation dataloaders.

In [ ]:
train_filters = dl.Filters(field="metadata.system.tags.train", values=True)
validation_filters = dl.Filters(field="metadata.system.tags.validation", values=True)

clone_model.dataset_id = dataset.id
clone_model.add_subset(subset_name="train", subset_filter=train_filters)
clone_model.add_subset(subset_name="validation", subset_filter=validation_filters)
clone_model.update(system_metadata=True)

execution = clone_model.train(service_config={
    "name": "resnet-train-evaluate",
    "runtime": {
        "podType": "regular-m",
        "concurrency": 1,
        "runnerImage": "gcr.io/viewo-g/piper/agent/runner/apps/torch-models:0.1.4",
        "autoscaler": {
            "type": "rabbitmq",
            "minReplicas": 0,
            "maxReplicas": 2,
            "queueLength": 100
        }
    }
})

Wait for the training to finish

In [ ]:
execution.wait()

### 5. Model Predict

First, get the spesifc item we want to perfrom prediction with our model

In [ ]:
item_filename = '/Black-grass/6.png'

item = dataset.items.get(filepath=item_filename)

Run the model predict method on the item.

Notice: Since the predict method expect to recive a list of item ids, we will add the item id to a list that will be sent to the method.

In [ ]:
clone_model.predict(item_ids=[item.id])

### 6. Model Evaluate

Notice: To evaluate a model we need to:
1. Get a dataset, that will be used for the evaluation.
2. Define the test filters, to build the test dataloader.

In [ ]:
test_filters = dl.Filters(field='metadata.system.tags.test', values=True)

clone_model.evaluate(dataset_id=dataset.id,
                     filters=test_filters)